<h4>I used Claude, Anthropic's model to help with the code portion of this assignment.</h4>
<h4>I used Copilot for help understanding these concepts.</h4>

(1) Homoscedasticity is one of the assumptions required to show that Ordinary Least Squares is BLUE. OLS will still be unbiased in the presence of heteroscedasticity, i.e., $\mathbb{E}[\hat{\beta}] = \beta$, in the presence of heteroscedasticity. However, heteroscedasticity increases the variance of the OLS estimator. Run a simulation that loops the OLS estimator over data with and without heteroscedasticity and then show that the slope estimates from the heteroscedastic data has higher variance. Note that $\operatorname{Var}(\hat{\beta})$ this is different than what I do in the video by just looking at one standard error. Note that depends on other things that you should try to hold as close to constant as possible in your simulation.

In [ ]:
# todo 

(2) Come up with an example of a linear estimator that is unbiased but has a bigger variance than the OLS estimator. You don not need to prove it has a higher variance, but you should be able to come up with an example that almost certainly has a higher variance.

In [ ]:
# todo

(3) The Likelihood Function for a linear regression under the assumption that the error terms are normally distributed with variance $\sigma^2$ is:
$$
L(\beta_{0},\beta_{1},\sigma^{2}\;\bigm|\;y_{i},x_{i})
=
(2\pi\,\sigma^{2})^{-\frac{n}{2}}
\exp\!\Biggl(
  -\frac{1}{2\,\sigma^{2}}
  \sum_{i=1}^{n}\bigl(y_{i}-\beta_{0}-\beta_{1}x_{i}\bigr)^{2}
\Biggr)
$$
explain the terms of the Likelihood function then prove that this likelihood function leads to the same estimator as OLS under the classic assumptions. I'm okay with you using mathematical software like Mathematica or AI, or any other resources to help with this problem. Explain when an analyst might choose to use Maximum Likelihood estimation for regression in the context of the assumptions.

In [ ]:
# todo 